In [0]:
import os
import pandas as pd
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics

In [0]:
df = pd.read_csv("crx.csv",na_values=['?'])
df.head(5)

In [0]:
df.isnull().any()

In [0]:
new_df = df.dropna(subset=["a2", "a14"])

In [0]:
new_df.head(5)

In [0]:
col = ['a9','a10','a12','a13']
for c in col:
    new_df[c] = new_df.apply(lambda x: ord(x[c])-96, axis=1)

In [0]:
new_df.head(5)

In [0]:
cols = ['s3','a8','a9','a10','a11','a12','a13','a15']
train = new_df[cols].values

In [0]:
target1 = new_df['a2'].values
target2 = new_df['a14'].values

In [0]:
print(train.shape)
print(target1.shape,target2.shape)

In [0]:
def model1():
    model=Sequential()
    model.add(Dense(64, activation='relu', input_dim=8)) 
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1)) 
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


def model2():
    model=Sequential()
    #model.add(Dense(64, activation='relu', input_dim=8)) 
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1)) 
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [0]:
model_1 = model1()
model_1.fit(train, target1, batch_size=200, epochs=100, shuffle=True)

In [0]:
model_2 = model2()
model_2.fit(train, target2, batch_size=50, epochs=100, shuffle=True)

In [0]:
a2 = df[df['a2'].isnull().values==True][cols]
a14 = df[df['a14'].isnull().values==True][cols]

col = ['a9','a10','a12','a13']
for c in col:
    a2[c] = a2.apply(lambda x: ord(x[c])-96, axis=1)
    a14[c] = a14.apply(lambda x: ord(x[c])-96, axis=1)

In [0]:
a2_val = model_1.predict(a2[cols].values)

In [0]:
a14_val = model_2.predict(a14[cols].values)

In [0]:
a2_null = list(df['a2'].loc[df['a2'].isnull().values==True].index)
a14_null = list(df['a14'].loc[df['a14'].isnull().values==True].index)

In [0]:
for i in range(len(a2_null)):
    df['a2'][a2_null[i]] = a2_val[i][0]
    
for i in range(len(a14_null)):
    df['a14'][a14_null[i]] = a14_val[i][0]

In [0]:
df.isnull().any()